In [1]:
import sys
import os
import pickle
import pandas as pd
import json
import datetime
import time
import copy
import numpy as np

py_file_location = '../PrivatePackages'
sys.path.append(os.path.abspath(py_file_location))
from sklearn.model_selection import train_test_split

import JiaoCheng
import NingXiang

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
curated_data = pd.read_parquet('../data/curated_data_std_below_1.parquet')
curated_data = curated_data[['date', 
                        'stock',
                        'trend',
                        'closePrice_lag_1', 
                        'log_ret_normalised_by_day_lag_1', 
                        'closePriceNorm_lag_1', 
                        'log_ret_lag_1', 
                        'mean_log_ret_lag_1',
                        'mean_closePrice_lag_1',
                        'mean_closePriceNorm_lag_1',
                        'mean_log_ret_normalised_by_day_lag_1', 
                        'pos_log_ret_lag_1',
                        'pos_closePrice_lag_1',
                        'pos_closePriceNorm_lag_1',
                        'pos_log_ret_normalised_by_day_lag_1']]
curated_data['date'] = pd.to_datetime(curated_data['date'])

In [3]:
date = [datetime.datetime(year=2022, month=12, day = 31) + datetime.timedelta(days=i) for i in range(500)]
val_date_start = date[250]
test_date_start = date[250-4]

In [4]:
curated_data['date'] = pd.to_datetime(curated_data['date'])
curated_data = curated_data.dropna()

train = curated_data[(curated_data['date'] < val_date_start)]
val = curated_data[(curated_data['date'] < test_date_start) & (curated_data['date'] >= val_date_start)]
test = curated_data[(curated_data['date'] >= test_date_start)]

In [5]:
def data_factory(data, lag):
    x_list, y_list = list(), list()

    for id, stock_data in data.groupby('stock'):
        stock_data.sort_values(by='date', inplace=True)
        stock_data.drop(['stock', 'date'], axis=1, inplace=True)
        stock_data.index = range(len(stock_data))

        for i in range(len(stock_data)):
            
            if i < lag-1:
                continue

            x = stock_data.iloc[i-lag+1:i+1].values.copy()
            x[-1:, 0] = 0 # mask last day's return
            y = stock_data.loc[i]['trend']
            
            x_list.append(x)
            y_list.append(y)
    
    return x_list, y_list

In [6]:
LAG = 5

train_x, train_y = data_factory(train, LAG)
val_x, val_y = data_factory(val, LAG)
test_x, test_y = data_factory(test, LAG)

In [7]:
from models.RecurrentNeuralNetwork_pt import LongShortTermMemoryClassifier_pt as LSTMC

params = {
    'recurrent_hidden_layer_embed_dim': 25, 
    'recurrent_n_hidden_layers': 1, 
    'bidirectional': False, 
    'dense_n_hidden_layers': 1, 
    'batch_size': 128, 
    'learning_rate': 0.01, 
    'dense_hidden_layer_embed_dim': 25, 
    'activation': 'sigmoid', 
    'dropout_prob': 0, 
    'batch_normalisation': False, 
    'dense_layer_type': 'Dense', 
    'attention_num_heads': 0,
    'grad_clip': 0,
    'num_epochs': 250,
    'loss_function': 'CrossEntropy',
    'verbose': True,
    'random_state': 19260817}

model = LSTMC(**params)

model.fit(train_x, train_y)

/Applications/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Applications/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <0F72FEF0-4DF1-3E8A-90BA-513122A1950F> /Applications/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <57D24B07-8B24-3888-A2B5-2B4C95434BA4> /Applications/anaconda3/lib/python3.9/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Epoch: 1


27it [00:01, 25.06it/s]


Epoch [1/250], Train Avg Loss: 0.6950 Train Accuracy (Metric) 0.495316
Epoch: 2


27it [00:00, 28.59it/s]


...

Epoch [248/250], Train Avg Loss: 0.6399 Train Accuracy (Metric) 0.630269
Epoch: 249


27it [00:01, 26.37it/s]


Epoch [249/250], Train Avg Loss: 0.6417 Train Accuracy (Metric) 0.629098
Epoch: 250


27it [00:00, 28.18it/s]

Epoch [250/250], Train Avg Loss: 0.6400 Train Accuracy (Metric) 0.628806


In [8]:
from sklearn.metrics import accuracy_score

for i in range(14):
    print(curated_data['stock'].unique()[i])
    print(accuracy_score(test_y[i*250:(i+1)*250], model.predict(test_x[i*250:(i+1)*250])))

1
0.532
3
0.684
8
0.532
10
0.688
12
0.74
13
0.448
14
0.66
19
0.692
20
0.572
28
0.584
30
0.604
37
0.632
44
0.48
46
0.628


In [13]:
pred_df = pd.DataFrame()

for i in range(14):
    if accuracy_score(test_y[i*250:(i+1)*250], model.predict(test_x[i*250:(i+1)*250])) > 0.7:
        tmp_df = pd.DataFrame({'stock': [curated_data['stock'].unique()[i] for j in range(250)], 'date': [i for i in range(250)], 'pred': model.predict(test_x[i*250:(i+1)*250])})
        pred_df = pd.concat([pred_df, tmp_df], axis=0)

pred_df

,stock,date,pred
0,12,0,-1.0
1,12,1,-1.0
2,12,2,-1.0
3,12,3,1.0
4,12,4,-1.0
...,...,...,...
245,12,245,1.0
246,12,246,1.0
247,12,247,1.0
248,12,248,1.0


In [14]:
pred_df.to_csv('../pred.csv', index = False)